# 🦍 ComfyUI Gorilla Generator
Access ComfyUI GUI directly in your browser!

In [ ]:
# Install ComfyUI
!pip install requests
!git clone https://github.com/comfyanonymous/ComfyUI.git
!cd ComfyUI && pip install -r requirements.txt

In [ ]:
# Download FLUX models
!cd ComfyUI && python download_flux_models.py

In [ ]:
# Complete ComfyUI + Pinggy Setup (Fixed Order)
# 1. Install system dependencies first
!apt-get update
!apt-get install -y wget aria2 libgl1-mesa-glx

# 2. Install Pinggy package
!pip install pinggy

# 3. NOW import pinggy (after installation)
import pinggy

# 4. Create tunnel BEFORE starting ComfyUI
tunnel1 = pinggy.start_tunnel(forwardto="localhost:8188")
print(f"🌐 ComfyUI will be accessible at: {tunnel1.urls}")

# 5. Start ComfyUI server
!python main.py --listen 0.0.0.0

In [ ]:
# Debug and Logging Setup
import logging
import sys
import subprocess
import time
from datetime import datetime

# Set up comprehensive logging
logging.basicConfig(
    level=logging.DEBUG,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.StreamHandler(sys.stdout),
        logging.FileHandler('comfyui_debug.log')
    ]
)

def log_command(cmd, description=""):
    """Run a command and log its output"""
    logging.info(f"Running: {cmd} - {description}")
    try:
        result = subprocess.run(cmd, shell=True, capture_output=True, text=True, timeout=300)
        logging.info(f"Exit code: {result.returncode}")
        if result.stdout:
            logging.info(f"STDOUT: {result.stdout}")
        if result.stderr:
            logging.error(f"STDERR: {result.stderr}")
        return result
    except subprocess.TimeoutExpired:
        logging.error(f"Command timed out: {cmd}")
        return None
    except Exception as e:
        logging.error(f"Command failed: {cmd} - Error: {e}")
        return None

# Log system info
logging.info("=== ComfyUI Debug Session Started ===")
logging.info(f"Timestamp: {datetime.now()}")
logging.info(f"Python version: {sys.version}")

# Check GPU availability
gpu_check = log_command("nvidia-smi", "Check GPU availability")
if gpu_check and gpu_check.returncode == 0:
    logging.info("✅ GPU detected")
else:
    logging.error("❌ GPU not detected or nvidia-smi failed")

# Check if ComfyUI directory exists
comfyui_check = log_command("ls -la ComfyUI/", "Check ComfyUI directory")
if comfyui_check and comfyui_check.returncode == 0:
    logging.info("✅ ComfyUI directory found")
else:
    logging.error("❌ ComfyUI directory not found")

print("🔍 Debug logging enabled. Check 'comfyui_debug.log' for detailed logs.")
